In [2]:
import os

from sagemaker import get_execution_role
from sagemaker.huggingface import HuggingFace
from sagemaker.inputs import TrainingInput

In [3]:
# role = os.environ("SM_EXECUTION_ROLE")
role = get_execution_role()

In [16]:
# hyperparameters sent by the client are passed as command-line arguments to the script.
hyperparameters={
    "num_train_epochs": 20,
    "per_device_training_batch_size": 32,
    "pretrained_model_name_or_path": "distilbert-base-cased"
}

In [11]:
# SageMaker metrics automatically parses training job logs for metrics and sends them
# to CloudWatch. If you want SageMaker to parse the logs, you must specify the metric’s
# name and a regular expression for SageMaker to use to find the metric.
metric_definitions = [
    {"Name": "train_runtime", "Regex": "train_runtime.*=\D*(.*?)$"},
    {"Name": "eval_accuracy", "Regex": "eval_accuracy.*=\D*(.*?)$"},
    {"Name": "eval_loss", "Regex": "eval_loss.*=\D*(.*?)$"},
]

In [17]:
huggingface_estimator = HuggingFace(
    entry_point="start.py",
    source_dir="./src",
    instance_type="ml.p2.xlarge",
    instance_count=1,
    role=role,
    transformers_version="4.26.0",
    pytorch_version="1.13.1",
    py_version="py39",
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
)

In [13]:
data_location = "s3://sagemaker-project-p-lo6kmrzwou9t/processed/sample/distilbert-base-cased"

In [14]:
# The input dictionary is keyed on channel name.
# If using multiple channels for training data, you can specify a dict mapping channel names to strings or TrainingInput() 
# objects or FileSystemInput() objects.
# https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework.fit
inputs = {
    "train": TrainingInput(s3_data=f"{data_location}/train/"),
    "test": TrainingInput(s3_data=f"{data_location}/test/"),
}

In [18]:
huggingface_estimator.fit(inputs=inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


Using provided s3_resource


INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-05-29-22-10-21-046


2023-05-29 22:10:21 Starting - Starting the training job...
2023-05-29 22:10:45 Starting - Preparing the instances for training.........
2023-05-29 22:12:18 Downloading - Downloading input data
2023-05-29 22:12:18 Training - Downloading the training image...........................
2023-05-29 22:16:45 Training - Training image download completed. Training in progress.....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-29 22:17:18,026 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-29 22:17:18,041 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-29 22:17:18,054 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-29 22:17:18,057 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-29 22:17:18,284 sagemaker-training-toolkit INFO     Installing d

UnexpectedStatusException: Error for Training job huggingface-pytorch-training-2023-05-29-22-10-21-046: Failed. Reason: ClientError: Artifact upload failed:Insufficient disk space

In [ ]:
1+1